<a href="https://colab.research.google.com/github/emello23/Alura-Gemini/blob/main/Agente_nova_saida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install pandas google-api-python-client google-generativeai


In [7]:
# --- Definição do Agente Montador de Roteiros ---

import google.generativeai as genai

class MontadorDeRoteiros:
    """
    Um agente responsável por organizar as atrações por dia, considerando proximidade,
    hospedagem como ponto de partida/chegada, e adicionando elementos para tornar o roteiro prazeroso e divertido.
    """
    def __init__(self, modelo_llm: genai.GenerativeModel):
        if modelo_llm is None:
            raise ValueError("O modelo LLM (Gemini) não pode ser None para este agente.")
        self.modelo = modelo_llm
        self.safety_settings = [ # Configurações de segurança para o Gemini
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

    def montar_roteiro(self, cidade: str, pais: str, atracoes: list, hospedagem: str, data_chegada: str, data_partida: str) -> dict:
        """
        Monta um roteiro diário otimizado para as atrações, considerando a hospedagem e datas.
        """
        from datetime import datetime
        import json # Import json inside the method as it's only used here

        delta = datetime.strptime(data_partida, "%Y-%m-%d") - datetime.strptime(data_chegada, "%Y-%m-%d")
        dias_viagem = delta.days + 1 # Inclui o dia de chegada e partida

        atracoes_str = json.dumps(atracoes, ensure_ascii=False, indent=2)

        prompt = f"""
Você é um especialista em planejamento de roteiros de viagem, focado em otimização e experiência do viajante.
Sua tarefa é criar um roteiro diário detalhado para uma viagem a {cidade}, {pais}, com base nas atrações fornecidas.
A viagem será de {data_chegada} a {data_partida}, totalizando {dias_viagem} dias.
A hospedagem principal é em: {hospedagem}.

Atrações disponíveis (formato JSON):
{atracoes_str}

Critérios para o Roteiro:
1.  **Otimização Geográfica:** Agrupe atrações próximas no mesmo dia para minimizar deslocamentos.
2.  **Fluxo Lógico:** Organize as visitas de forma sequential e eficiente.
3.  **Ritmo Agradável:** Considere tempos de deslocamento, duração estimada das visitas e pausas (almoço, descanso). Evite sobrecarregar os dias.
4.  **Hospedagem como Base:** O roteiro deve idealmente começar e terminar o dia próximo à área da hospedagem, se possível.
5.  **Flexibilidade:** Mencione que o roteiro é uma sugestão e pode ser adaptado.
6.  **Sugestões Adicionais:** Inclua sugestões gerais de onde comer na região das atrações do dia, ou atividades noturnas, se apropriado para a cidade.
7.  **Formato Diário:** Organize a saída por dia da viagem.

Formato da Resposta:
Retorne **estritamente uma lista JSON**. Cada elemento da lista deve ser um dicionário representando um dia do roteiro,
contendo EXATAMENTE as seguintes chaves:
- "dia": (integer) O número do dia da viagem (começando em 1).
- "data": (string) A data correspondente ao dia do roteiro no formato YYYY-MM-DD.
- "atividades_sugeridas": (lista de strings) Uma lista ordenada das atrações e atividades sugeridas para este dia, com descrições concisas (ex: "Visitar o Museu do Louvre (reservar 3-4 horas)", "Passeio pelo Jardim de Tuileries (1-2 horas)", "Almoço na região do museu"). Inclua tempos estimados ou sugestões de duração quando possível.
- "observacoes": (string) Sugestões adicionais para o dia, como opções de transporte, áreas para almoço/jantar, ou dicas específicas para as atrações do dia. Use "N/A" se não houver observações relevantes.

Exemplo de um item na lista JSON:
{{
  "dia": 1,
  "data": "{data_chegada}",
  "atividades_sugeridas": [
    "Chegada e check-in na hospedagem ({hospedagem})",
    "Passeio exploratório pela vizinhança da hospedagem",
    "Jantar em um restaurante local"
  ],
  "observacoes": "Aproveite para se ambientar na área."
}}

Garanta que a saída seja SOMENTE a lista JSON, sem nenhum texto introdutório, comentários ou formatação adicional.
"""

        display(Markdown(f"🧠 *Agente montando roteiro para: **{cidade}, {pais}** ({dias_viagem} dias)...*"))

        try:
            response = self.modelo.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.7, # Um pouco mais criativo para o roteiro
                    max_output_tokens=4096 # Espaço suficiente para o roteiro completo
                ),
                safety_settings=self.safety_settings
            )

            roteiro_text = response.text.strip()
            # Tentativa robusta de extrair JSON
            if roteiro_text.startswith("```json"):
                roteiro_text = roteiro_text[7:]
            if roteiro_text.endswith("```"):
                roteiro_text = roteiro_text[:-3]
            roteiro_text = roteiro_text.strip()


            if not roteiro_text:
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou uma resposta vazia ao tentar montar o roteiro para {cidade}.</font>"))
                 return {"erro": "resposta vazia do Gemini"}


            roteiro = json.loads(roteiro_text)
            if not isinstance(roteiro, list):
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou um formato inesperado (não uma lista) para o roteiro de {cidade}.</font>"))
                 return {"erro": "formato inesperado do Gemini"}

            display(Markdown(f"✅ *Gemini montou um roteiro com {len(roteiro)} dias para {cidade}.*"))
            return {"roteiro": roteiro}

        except json.JSONDecodeError as e:
            display(Markdown(f"<font color='red'>**Erro (JSONDecodeError) ao processar resposta do Gemini para o roteiro de {cidade}:** {e}. "
                             "Isso geralmente ocorre se o modelo não retornar um JSON válido.</font>"))
            if hasattr(response, 'text'):
                display(Markdown(f"<pre>Resposta Bruta do Gemini:\n{response.text}</pre>"))
            else:
                 display(Markdown(f"<pre>Nenhuma resposta de texto recebida do Gemini.</pre>"))
            return {"erro": f"JSONDecodeError: {e}"}
        except Exception as e:
            display(Markdown(f"<font color='red'>**Erro inesperado ao consultar Gemini para montar o roteiro de {cidade}:** {e}</font>"))
            if hasattr(response, 'prompt_feedback'):
                display(Markdown(f"Feedback do Prompt: {response.prompt_feedback}"))
            return {"erro": f"Erro inesperado: {e}"}

In [8]:
# --- Definição do Agente Montador de Roteiros ---

class MontadorDeRoteiros:
    """
    Um agente responsável por organizar as atrações por dia, considerando proximidade,
    hospedagem como ponto de partida/chegada, e adicionando elementos para tornar o roteiro prazeroso e divertido.
    """
    def __init__(self, modelo_llm: genai.GenerativeModel):
        if modelo_llm is None:
            raise ValueError("O modelo LLM (Gemini) não pode ser None para este agente.")
        self.modelo = modelo_llm
        self.safety_settings = [ # Configurações de segurança para o Gemini
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

    def montar_roteiro(self, cidade: str, pais: str, atracoes: list, hospedagem: str, data_chegada: str, data_partida: str) -> dict:
        """
        Monta um roteiro diário otimizado para as atrações, considerando a hospedagem e datas.
        """
        from datetime import datetime
        delta = datetime.strptime(data_partida, "%Y-%m-%d") - datetime.strptime(data_chegada, "%Y-%m-%d")
        dias_viagem = delta.days + 1 # Inclui o dia de chegada e partida

        atracoes_str = json.dumps(atracoes, ensure_ascii=False, indent=2)

        prompt = f"""
        Você é um especialista em planejamento de roteiros de viagem, focado em otimização e experiência do viajante.
        Sua tarefa é criar um roteiro diário detalhado para uma viagem a {cidade}, {pais}, com base nas atrações fornecidas.
        A viagem será de {data_chegada} a {data_partida}, totalizando {dias_viagem} dias.
        A hospedagem principal é em: {hospedagem}.

        Atrações disponíveis (formato JSON):
{atracoes_str}

        Critérios para o Roteiro:
        1.  **Otimização Geográfica:** Agrupe atrações próximas no mesmo dia para minimizar deslocamentos.
        2.  **Fluxo Lógico:** Organize as visitas de forma sequencial e eficiente.
        3.  **Ritmo Agradável:** Considere tempos de deslocamento, duração estimada das visitas e pausas (almoço, descanso). Evite sobrecarregar os dias.
        4.  **Hospedagem como Base:** O roteiro deve idealmente começar e terminar o dia próximo à área da hospedagem, se possível.
        5.  **Flexibilidade:** Mencione que o roteiro é uma sugestão e pode ser adaptado.
        6.  **Sugestões Adicionais:** Inclua sugestões gerais de onde comer na região das atrações do dia, ou atividades noturnas, se apropriado para a cidade.
        7.  **Formato Diário:** Organize a saída por dia da viagem.

        Formato da Resposta:
        Retorne **estritamente uma lista JSON**. Cada elemento da lista deve ser um dicionário representando um dia do roteiro,
        contendo EXATAMENTE as seguintes chaves:
        - "dia": (integer) O número do dia da viagem (começando em 1).
        - "data": (string) A data correspondente ao dia do roteiro no formato YYYY-MM-DD.
        - "atividades_sugeridas": (lista de strings) Uma lista ordenada das atrações e atividades sugeridas para este dia, com descrições concisas (ex: "Visitar o Museu do Louvre (reservar 3-4 horas)", "Passeio pelo Jardim de Tuileries (1-2 horas)", "Almoço na região do museu"). Inclua tempos estimados ou sugestões de duração quando possível.
        - "observacoes": (string) Sugestões adicionais para o dia, como opções de transporte, áreas para almoço/jantar, ou dicas específicas para as atrações do dia. Use "N/A" se não houver observações relevantes.

        Exemplo de um item na lista JSON:
        {{
          "dia": 1,
          "data": "{data_chegada}",
          "atividades_sugeridas": [
            "Chegada e check-in na hospedagem ({hospedagem})",
            "Passeio exploratório pela vizinhança da hospedagem",
            "Jantar em um restaurante local"
          ],
          "observacoes": "Aproveite para se ambientar na área."
        }}

        Garanta que a saída seja SOMENTE a lista JSON, sem nenhum texto introdutório, comentários ou formatação adicional.
        """

        display(Markdown(f"🧠 *Agente montando roteiro para: **{cidade}, {pais}** ({dias_viagem} dias)...*"))

        try:
            response = self.modelo.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.7, # Um pouco mais criativo para o roteiro
                    max_output_tokens=4096 # Espaço suficiente para o roteiro completo
                ),
                safety_settings=self.safety_settings
            )

            roteiro_text = response.text.strip()
            # Tentativa robusta de extrair JSON
            if roteiro_text.startswith("```json"):
                roteiro_text = roteiro_text[7:]
            if roteiro_text.endswith("```"):
                roteiro_text = roteiro_text[:-3]
            roteiro_text = roteiro_text.strip()


            if not roteiro_text:
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou uma resposta vazia ao tentar montar o roteiro para {cidade}.</font>"))
                 return {"erro": "resposta vazia do Gemini"}


            roteiro = json.loads(roteiro_text)
            if not isinstance(roteiro, list):
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou um formato inesperado (não uma lista) para o roteiro de {cidade}.</font>"))
                 return {"erro": "formato inesperado do Gemini"}

            display(Markdown(f"✅ *Gemini montou um roteiro com {len(roteiro)} dias para {cidade}.*"))
            return {"roteiro": roteiro}

        except json.JSONDecodeError as e:
            display(Markdown(f"<font color='red'>**Erro (JSONDecodeError) ao processar resposta do Gemini para o roteiro de {cidade}:** {e}. "
                             "Isso geralmente ocorre se o modelo não retornar um JSON válido.</font>"))
            if hasattr(response, 'text'):
                display(Markdown(f"<pre>Resposta Bruta do Gemini:\n{response.text}</pre>"))
            else:
                 display(Markdown(f"<pre>Nenhuma resposta de texto recebida do Gemini.</pre>"))
            return {"erro": f"JSONDecodeError: {e}"}
        except Exception as e:
            display(Markdown(f"<font color='red'>**Erro inesperado ao consultar Gemini para montar o roteiro de {cidade}:** {e}</font>"))
            if hasattr(response, 'prompt_feedback'):
                display(Markdown(f"Feedback do Prompt: {response.prompt_feedback}"))
            return {"erro": f"Erro inesperado: {e}"}

In [9]:
# --- Definição do Agente Montador de Roteiros ---

class MontadorDeRoteiros:
    """
    Um agente responsável por organizar as atrações por dia, considerando proximidade,
    hospedagem como ponto de partida/chegada, e adicionando elementos para tornar o roteiro prazeroso e divertido.
    """
    def __init__(self, modelo_llm: genai.GenerativeModel):
        if modelo_llm is None:
            raise ValueError("O modelo LLM (Gemini) não pode ser None para este agente.")
        self.modelo = modelo_llm
        self.safety_settings = [ # Configurações de segurança para o Gemini
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

    def montar_roteiro(self, cidade: str, pais: str, atracoes: list, hospedagem: str, data_chegada: str, data_partida: str) -> dict:
        """
        Monta um roteiro diário otimizado para as atrações, considerando a hospedagem e datas.
        """
        from datetime import datetime
        delta = datetime.strptime(data_partida, "%Y-%m-%d") - datetime.strptime(data_chegada, "%Y-%m-%d")
        dias_viagem = delta.days + 1 # Inclui o dia de chegada e partida

        atracoes_str = json.dumps(atracoes, ensure_ascii=False, indent=2)

        prompt = f"""
        Você é um especialista em planejamento de roteiros de viagem, focado em otimização e experiência do viajante.
        Sua tarefa é criar um roteiro diário detalhado para uma viagem a {cidade}, {pais}, com base nas atrações fornecidas.
        A viagem será de {data_chegada} a {data_partida}, totalizando {dias_viagem} dias.
        A hospedagem principal é em: {hospedagem}.

        Atrações disponíveis (formato JSON):
{atracoes_str}

        Critérios para o Roteiro:
        1.  **Otimização Geográfica:** Agrupe atrações próximas no mesmo dia para minimizar deslocamentos.
        2.  **Fluxo Lógico:** Organize as visitas de forma sequencial e eficiente.
        3.  **Ritmo Agradável:** Considere tempos de deslocamento, duração estimada das visitas e pausas (almoço, descanso). Evite sobrecarregar os dias.
        4.  **Hospedagem como Base:** O roteiro deve idealmente começar e terminar o dia próximo à área da hospedagem, se possível.
        5.  **Flexibilidade:** Mencione que o roteiro é uma sugestão e pode ser adaptado.
        6.  **Sugestões Adicionais:** Inclua sugestões gerais de onde comer na região das atrações do dia, ou atividades noturnas, se apropriado para a cidade.
        7.  **Formato Diário:** Organize a saída por dia da viagem.

        Formato da Resposta:
        Retorne **estritamente uma lista JSON**. Cada elemento da lista deve ser um dicionário representando um dia do roteiro,
        contendo EXATAMENTE as seguintes chaves:
        - "dia": (integer) O número do dia da viagem (começando em 1).
        - "data": (string) A data correspondente ao dia do roteiro no formato YYYY-MM-DD.
        - "atividades_sugeridas": (lista de strings) Uma lista ordenada das atrações e atividades sugeridas para este dia, com descrições concisas (ex: "Visitar o Museu do Louvre (reservar 3-4 horas)", "Passeio pelo Jardim de Tuileries (1-2 horas)", "Almoço na região do museu"). Inclua tempos estimados ou sugestões de duração quando possível.
        - "observacoes": (string) Sugestões adicionais para o dia, como opções de transporte, áreas para almoço/jantar, ou dicas específicas para as atrações do dia. Use "N/A" se não houver observações relevantes.

        Exemplo de um item na lista JSON:
        {{
          "dia": 1,
          "data": "{data_chegada}",
          "atividades_sugeridas": [
            "Chegada e check-in na hospedagem ({hospedagem})",
            "Passeio exploratório pela vizinhança da hospedagem",
            "Jantar em um restaurante local"
          ],
          "observacoes": "Aproveite para se ambientar na área."
        }}

        Garanta que a saída seja SOMENTE a lista JSON, sem nenhum texto introdutório, comentários ou formatação adicional.
        """

        display(Markdown(f"🧠 *Agente montando roteiro para: **{cidade}, {pais}** ({dias_viagem} dias)...*"))

        try:
            response = self.modelo.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.7, # Um pouco mais criativo para o roteiro
                    max_output_tokens=4096 # Espaço suficiente para o roteiro completo
                ),
                safety_settings=self.safety_settings
            )

            roteiro_text = response.text.strip()
            # Tentativa robusta de extrair JSON
            if roteiro_text.startswith("```json"):
                roteiro_text = roteiro_text[7:]
            if roteiro_text.endswith("```"):
                roteiro_text = roteiro_text[:-3]
            roteiro_text = roteiro_text.strip()


            if not roteiro_text:
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou uma resposta vazia ao tentar montar o roteiro para {cidade}.</font>"))
                 return {"erro": "resposta vazia do Gemini"}


            roteiro = json.loads(roteiro_text)
            if not isinstance(roteiro, list):
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou um formato inesperado (não uma lista) para o roteiro de {cidade}.</font>"))
                 return {"erro": "formato inesperado do Gemini"}

            display(Markdown(f"✅ *Gemini montou um roteiro com {len(roteiro)} dias para {cidade}.*"))
            return {"roteiro": roteiro}

        except json.JSONDecodeError as e:
            display(Markdown(f"<font color='red'>**Erro (JSONDecodeError) ao processar resposta do Gemini para o roteiro de {cidade}:** {e}. "
                             "Isso geralmente ocorre se o modelo não retornar um JSON válido.</font>"))
            if hasattr(response, 'text'):
                display(Markdown(f"<pre>Resposta Bruta do Gemini:\n{response.text}</pre>"))
            else:
                 display(Markdown(f"<pre>Nenhuma resposta de texto recebida do Gemini.</pre>"))
            return {"erro": f"JSONDecodeError: {e}"}
        except Exception as e:
            display(Markdown(f"<font color='red'>**Erro inesperado ao consultar Gemini para montar o roteiro de {cidade}:** {e}</font>"))
            if hasattr(response, 'prompt_feedback'):
                display(Markdown(f"Feedback do Prompt: {response.prompt_feedback}"))
            return {"erro": f"Erro inesperado: {e}"}

In [10]:
dados_finais_viagem = dados_compilados = executar_planejador_viagem("/content/europa.csv")

NameError: name 'executar_planejador_viagem' is not defined

In [ ]:
if dados_finais_viagem and dados_finais_viagem[0].get("roteiro"):
    display(Markdown("### Roteiro Sugerido para a Primeira Cidade:"))
    for dia in dados_finais_viagem[0]["roteiro"]:
         display(Markdown(f"**Dia {dia['dia']} ({dia['data']}):**"))
         for atividade in dia['atividades_sugeridas']:
             display(Markdown(f"- {atividade}"))
         if dia.get("observacoes") and dia["observacoes"] != "N/A":
             display(Markdown(f"*Observações:* {dia['observacoes']}"))
         display(Markdown("---"))

if dados_finais_viagem and dados_finais_viagem[0].get("pesquisa") and dados_finais_viagem[0]["pesquisa"].get("sugestoes_gemini"):
     display(Markdown("### Sugestões de Atrações do Gemini para a Primeira Cidade:"))
     for atracao in dados_finais_viagem[0]["pesquisa"]["sugestoes_gemini"]:
          display(Markdown(f"- **{atracao['nome']}** ({atracao['tipo_principal']}): {atracao['descricao_enxuta']} (Avaliação: {atracao.get('avaliacao_geral', 'N/A')})"))

In [ ]:
dados_finais_viagem = dados_compilados = executar_planejador_viagem("/content/europa.csv")

In [ ]:
if dados_finais_viagem and dados_finais_viagem[0].get("roteiro"):
    display(Markdown("### Roteiro Sugerido para a Primeira Cidade:"))
    for dia in dados_finais_viagem[0]["roteiro"]:
         display(Markdown(f"**Dia {dia['dia']} ({dia['data']}):**"))
         for atividade in dia['atividades_sugeridas']:
             display(Markdown(f"- {atividade}"))
         if dia.get("observacoes") and dia["observacoes"] != "N/A":
             display(Markdown(f"*Observações:* {dia['observacoes']}"))
         display(Markdown("---"))

if dados_finais_viagem and dados_finais_viagem[0].get("pesquisa") and dados_finais_viagem[0]["pesquisa"].get("sugestoes_gemini"):
     display(Markdown("### Sugestões de Atrações do Gemini para a Primeira Cidade:"))
     for atracao in dados_finais_viagem[0]["pesquisa"]["sugestoes_gemini"]:
          display(Markdown(f"- **{atracao['nome']}** ({atracao['tipo_principal']}): {atracao['descricao_enxuta']} (Avaliação: {atracao.get('avaliacao_geral', 'N/A')})"))